# Demo of angle prediction


##### 1. Import all necessary libraries and functions


In [3]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from model import get_model
from preprocess_data import get_data
from main import load_image, Racoon
from dataset_creation import change_filepath_in_json, get_list_of_possible_angles, process_images

#### 2. Create dataset from initial images

In [4]:
change_filepath_in_json()
get_list_of_possible_angles("dataset/formatted_initial_data.json")
process_images("dataset/angles_data.json")

Created images for initial_data/images/0.png
Created images for initial_data/images/1.png
Created images for initial_data/images/2.png
Created images for initial_data/images/3.png
Created images for initial_data/images/4.png
Created images for initial_data/images/5.png
Created images for initial_data/images/6.png
Created images for initial_data/images/7.png
Created images for initial_data/images/8.png
Created images for initial_data/images/9.png
Created images for initial_data/images/10.png


#### 3. Train your own model **(optional)**


#### 3.1. Load the data

In [5]:
x, y = get_data(0, 670)
x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size=0.05, random_state=42
)
x.shape, x_val.shape

((670, 200, 400, 3), (34, 200, 400, 3))

#### 3.2. Train model


In [ ]:
model = get_model()

# Compile the model
optimizer = Adam(learning_rate=0.0001, clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer, metrics=["mae"])

# Define callbacks
checkpoint = ModelCheckpoint(
    "best_model.h5", monitor="val_loss", verbose=1, save_best_only=True, mode="min"
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6
)
early_stopping = EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

# Train the model with callbacks
history = model.fit(
    x_train,
    y_train,
    shuffle=True,
    batch_size=64,
    epochs=50,
    validation_data=(x_val, y_val),
    callbacks=[checkpoint, reduce_lr, early_stopping],
)

# Load the best model weights
model.load_weights("best_model.h5")

tf.saved_model.save(model, "saved_model")


#### 4. Get prediction


In [20]:
# Define the image path
image_path = "initial_data/images/6.png"

# Create an instance of the Racoon class
racoon = Racoon()
image = load_image(image_path)
if image:
    racoon.tranform(image, True)

Predicted angle: 2.54


Initial image:
<img src="initial_data/images/6.png" height=700 width=350>

Image after rotation:
<img src="initial_data/images/6_result.png" height=700 width=350>

In [22]:
# Define the image path
image_path = "initial_data/images/4.png"

# Create an instance of the Racoon class
racoon = Racoon()
image = load_image(image_path)
if image:
    racoon.tranform(image, True)

Predicted angle: -0.26


Initial image:
<img src="initial_data/images/4.png" height=700 width=350>

Image after rotation:
<img src="initial_data/images/4_result.png" height=700 width=350>